# Read VTK files and return a csv with time series of wake centers
Just tested with YZ planes. Probably needs modifications for other directions.

**Inputs:** Given in the second cell

**Outputs:** CSV with columns time, xc, yc, zc. Nee


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyFAST.input_output.vtk_file as vtk_file
from pyFAST.input_output import TurbSimFile
from samwich.waketrackers import track
from samwich.dataloaders import PlanarData

from scipy import interpolate

def interpolate_grid(M, y, z, y_new, z_new, method='linear'):        
    # Create an interpolation function
    interp_func = interpolate.RectBivariateSpline(y, z, M)

    # Interpolate the values to the new grid
    M_new = interp_func(y_new, z_new)

    return M_new

In [ ]:
# Temporary inputs - Used only to make the specification of file names easier
seed = 0
seaState = 'mild'
condition = 'Cond09_v11.4_PL0.02_TI5'

# Main inputs
dt = 3       # Time step between vtk files
folderPath = f'/scratch/ldocarm/WakesFloating/3turbine-farm/{seaState}/{condition}/Case0_wdirp00/Seed_{seed}' # Path to the folder with the FAST.Farm simulation
rootNameVTK = 'FFarm_mod.Low.DisYZ'     # Root name of the test case
tsFilePath  = f'/scratch/ldocarm/WakesFloating/3turbine-farm/{seaState}/{condition}/Seed_{seed}/Low.bts' # Path to the binary TurbSim file used as input to the simulation that generated those vtk's
outFolder = os.path.join(folderPath, 'wakesFromSAMWICH') # Output folder. Output files will have the same name as the vtks, but with .csv extensions.
rotorDiam = 126 # Rotor diameter
rootPath = os.path.join(folderPath, 'vtk_ff') # Subfolder with vtk's generated by FAST.Farm
zHub = 180.25 # Hub height in the turbsim file for computing the advective velocity


listXPos = [-240.0, 126.0, 252.0, 378.0, 504.0, 630.0, 756.0, 1638.0, 2520.0] # X positions of the planes that will be processed. Need to be the same length as the number of planes with vtk files
numPlans = len(listXPos)


# Create the output folder if it does not exist
if not os.path.exists(outFolder):
    os.makedirs(outFolder)

In [ ]:
# Need to read TurbSim to remove the undisturbed flow
ts = TurbSimFile(tsFilePath)
u_T = np.mean(ts['u'][0,:, :, :], axis=0) # Time average
u_YT = np.mean(u_T, axis=0) # Average over y after time average

iz = np.argmin(np.abs(ts['z']-zHub))
u_adv = u_YT[iz]
print(f"u_adv={u_adv}")
print(ts.keys())
print(ts['y'].shape)

In [ ]:
# List all vtk files
listNames = []
for i in range(numPlans):
    # Some versions of FAST.Farm write VTk names as 01 and others 1
    nameVTK = f'{rootNameVTK}{i+1:02d}'
    nameVTK_wo0 = f'{rootNameVTK}{i+1}'

    # List all files with nameVTK in the name
    thisListOfNames = [f for f in os.listdir(rootPath) if os.path.isfile(os.path.join(rootPath, f)) and nameVTK in f or nameVTK_wo0 in f]
    listNames.extend(thisListOfNames)

list_vtk = []
for vtkName in listNames:
    list_vtk.append(vtk_file.VTKFile(rootPath + "/" + vtkName))

# Check how many files
print(len(list_vtk))

In [ ]:
# Compute wake centers with SAMWICH and time series to a csv file 

# Sort listNames alphabetically. That is enough to make sure that things are sorted by time
listNames = sorted(listNames)

for i in range(numPlans):
# for i in range(5,6):
    # Get indices of files for this plan
    ind4plan = [j for j, nm in enumerate(listNames) if f'{rootNameVTK}{i+1:02d}' in nm or f'{rootNameVTK}{i+1}' in nm]
    # Write to a csv file
    outFlPath = os.path.join(outFolder, f'{rootNameVTK}{i+1:02d}.csv')    

    with open(outFlPath, 'w') as f:
        f.write('t,x,y,z\n')        

        dict4wake = {'y':[], 'z':[], 'u':[], 'v':[], 'w':[]}   # Used by SAMWICH
        for iTime, iP in enumerate(ind4plan):            
#             if iTime < 573 or iTime > 573:
#                 continue
            time = dt*int(listNames[iP].split('.')[-2]) # TIme
            vtk = list_vtk[iP]            

            # 2D arrays with shape len(vtk.yp_grid): rows, len(vtk.zp_grid): columns
            dict4wake['u'] = vtk.point_data_grid["Velocity"][0, :, :, 0]
            dict4wake['v'] = vtk.point_data_grid["Velocity"][0, :, :, 1]
            dict4wake['w'] = vtk.point_data_grid["Velocity"][0, :, :, 2]
            xc = listXPos[i] # X position of the wake center (constante and equal to plane position)

            # Need 2D arrays with same shape as u, v, w
            dict4wake['z'], dict4wake['y'] = np.meshgrid(vtk.zp_grid, vtk.yp_grid)

            wakedata = PlanarData(dict4wake)
            wake = track(wakedata.sliceI(), method='ConstantArea', verbose=False)

            # Compute wake centers
            refArea = np.pi*rotorDiam**2/4
            
            # Use this if going to remove the time and spatial average
#             free_Uprofile = np.interp(wake.z, ts['z'], u_YT)

            # Use this to use the time varying inflow
            # First, find time index in ts. Does not need to be exact, but should account for downstream distance
            ind4ts = int((time-xc/u_adv)/ts['dt'])
            if ind4ts >= ts['u'].shape[1]: # Lazy fix for a rounding error in last time step
                ind4ts = ts['u'].shape[1]-1
            free_Uprofile = interpolate_grid(ts['u'][0,ind4ts,:,:], ts['y'], ts['z'], wake.y[:,0], wake.z[0,:])            
            
            wake.remove_shear(wind_profile=free_Uprofile)
            wake.wake_tracked = False  # force recalculation for python notebook                
            # outMat[iTime, 2], outMat[iTime, 3] = wake.find_centers(refArea, weighted_center=False) # geometric center
            # outMat[iTime, 2], outMat[iTime, 3] = wake.find_centers(refArea, weighted_center=True)  # weighted by velocity deficit
            yc, zc = wake.find_centers(refArea, weighted_center=lambda u: u**2) # y and z coordinates of the wake center
#             yc, zc = wake.find_centers(umin=None,  sigma=rotorDiam)
            
            # Write to file
            f.write(f'{time:.3f},{xc:.3f},{yc[0]:.3f},{zc[0]:.3f}\n')
            f.flush()            
print("Done!")